# **Main FNN Pruned**

In [ ]:
"""
FNN-only pruning pipeline (auto-detect model type).
- Detects model type from final layer (binary / multiclass / regression).
- Adapts CSV loader accordingly.
- Keeps final layer intact (never pruned).
- Reports FLOPs, GFLOPs, MFLOPS and inference time.
"""

import numpy as np
import pandas as pd
import tensorflow as tf
import time
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
import os

# -----------------------
# Helper: detect model type
# -----------------------
def detect_model_type(model):
    out = model.layers[-1]
    units = getattr(out, "units", None)
    act = getattr(out, "activation", None)
    act_name = act.__name__ if act is not None else None

    if units == 1 and act_name in ("sigmoid",):
        return "binary"
    if units == 1 and act_name in ("linear", "relu", "tanh"):
        return "regression"
    if units is not None and units > 1 and act_name == "softmax":
        return "multiclass"
    # fallback tries
    if units == 1:
        return "binary"
    if units and units > 1:
        return "multiclass"
    return "unknown"

def get_loss_and_metrics_for_type(model_type):
    if model_type == "binary":
        return ("binary_crossentropy", ["accuracy"])
    if model_type == "multiclass":
        # use sparse labels to avoid forcing one-hot encoding
        return ("sparse_categorical_crossentropy", ["sparse_categorical_accuracy"])
    if model_type == "regression":
        return ("mse", ["mse"])
    raise ValueError("Unknown model type")

# =====================================================================
# 1) UNIVERSAL CSV LOADER  (adapts target based on model_type)
# =====================================================================
def load_any_csv_dataset(csv_path, target=None, test_size=0.2, val_size=0.1, model_type="binary"):
    """
    Loads CSV (auto-detect delimiter). Adapts target processing to model_type:
      - binary/regression: returns y as float32 shaped (n,1)
      - multiclass: returns integer labels shaped (n,1) for sparse loss
    """
    print("[INFO] Loading CSV with automatic delimiter detection...")
    df = pd.read_csv(csv_path, sep=None, engine="python")
    print("[INFO] Columns detected:", list(df.columns))

    if target is None:
        target = df.columns[-1]
        print(f"[INFO] Auto-detected target column = {target}")

    if target not in df.columns:
        raise ValueError(f"Target column '{target}' not found in dataset!")

    y = df[target]
    X = df.drop(columns=[target])

    # Preprocess target based on model_type
    if model_type == "binary":
        # convert yes/no/true/false -> 1/0
        if y.dtype == object:
            y = y.astype(str).str.strip().str.lower()
            y = y.replace({"yes": 1, "no": 0, "true": 1, "false": 0})
        # ensure numeric
        y = y.astype(np.float32).values.reshape(-1, 1)

    elif model_type == "multiclass":
        # convert strings to categorical integer codes if needed
        if y.dtype == object:
            y = y.astype("category").cat.codes
        # ensure integer labels (sparse labels)
        y = y.astype(np.int32).values.reshape(-1, 1)

    elif model_type == "regression":
        # numeric continuous
        y = y.astype(np.float32).values.reshape(-1, 1)

    else:
        raise ValueError("Unsupported model_type for loader")

    # One-hot encode only categorical predictors (keep numeric as-is)
    cat_cols = X.select_dtypes(include=["object"]).columns
    if len(cat_cols) > 0:
        print("[INFO] One-hot encoding predictors:", list(cat_cols))
        X = pd.get_dummies(X, drop_first=True)
    else:
        print("[INFO] No categorical predictor columns to encode.")

    # convert to float32 (features)
    X = X.astype(np.float32)

    # Split
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=(test_size + val_size), random_state=42, stratify=y if model_type!="regression" else None
    )
    # second split into val/test
    val_ratio = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=(1 - val_ratio), random_state=42,
        stratify=y_temp if model_type!="regression" else None
    )

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

# =====================================================================
# 2) GET DENSE LAYERS
# =====================================================================
def get_dense_layers(model):
    return [layer for layer in model.layers if isinstance(layer, Dense)]

# =====================================================================
# 3) ACTIVATION + GRADIENT STATS  (SHAPE-SAFE)
# =====================================================================
def compute_activation_grad_stats(model, dense_layers, X, y, model_type,
                                  batch_size=64, max_batches=30):
    """
    Returns stats dict: {layer.name: (A_mean_per_neuron, G_mean_per_neuron, Var_per_neuron)}
    Uses appropriate loss for model_type when computing gradients.
    """
    # Convert X,y to numpy if DataFrame
    if isinstance(X, pd.DataFrame):
        X_np = X.to_numpy()
    else:
        X_np = X
    if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
        y_np = y.to_numpy()
    else:
        y_np = y

    ds = tf.data.Dataset.from_tensor_slices((X_np, y_np)).batch(batch_size)
    acc_A = {l.name: [] for l in dense_layers}
    acc_G = {l.name: [] for l in dense_layers}
    acc_V = {l.name: [] for l in dense_layers}

    # choose loss for gradient computation
    if model_type == "binary":
        loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    elif model_type == "multiclass":
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction=tf.keras.losses.Reduction.NONE)
    elif model_type == "regression":
        loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    else:
        raise ValueError("Unknown model_type for stats")

    batch_count = 0
    for xb, yb in ds:
        if batch_count >= max_batches:
            break
        batch_count += 1

        # ensure shapes consistent
        x = tf.cast(xb, tf.float32)
        # For sparse multiclass loss, yb should be shape (batch,) ints
        if model_type == "multiclass":
            yb_proc = tf.cast(tf.squeeze(yb, axis=-1), tf.int32)
        else:
            # binary/regression: keep shape (batch,1) as float32
            yb_proc = tf.cast(yb, tf.float32)

        layer_outputs = {}
        with tf.GradientTape(persistent=True) as tape:
            for layer in model.layers:
                x = layer(x)
                if layer in dense_layers:
                    tape.watch(x)
                    layer_outputs[layer.name] = x
            preds = x
            # compute per-sample loss (shape [batch,] or [batch, ...])
            per_sample_loss = loss_fn(yb_proc, preds)
            # compute scalar loss for gradient direction
            loss = tf.reduce_mean(per_sample_loss)

        for layer in dense_layers:
            name = layer.name
            if name not in layer_outputs:
                continue
            a = layer_outputs[name]            # shape (batch, units)
            A = tf.reduce_mean(tf.abs(a), axis=0).numpy()
            V = tf.math.reduce_variance(a, axis=0).numpy()

            g = tape.gradient(loss, a)
            if g is None:
                G = np.zeros_like(A)
            else:
                G = tf.reduce_mean(tf.abs(g), axis=0).numpy()

            acc_A[name].append(A)
            acc_G[name].append(G)
            acc_V[name].append(V)

        del tape

    # Aggregate
    stats = {}
    for layer in dense_layers:
        name = layer.name
        if len(acc_A[name]) == 0:
            stats[name] = (np.array([]), np.array([]), np.array([]))
            continue
        A = np.mean(np.stack(acc_A[name], axis=0), axis=0)
        G = np.mean(np.stack(acc_G[name], axis=0), axis=0)
        V = np.mean(np.stack(acc_V[name], axis=0), axis=0)
        stats[name] = (A, G, V)

    return stats

# =====================================================================
# 4) IMPORTANCE MASKS (protect last Dense layer)
# =====================================================================
def compute_importance_mask(
        stats,
        keep_ratio=0.7,
        alpha=0.5,
        beta=0.3,
        gamma=0.2,
        last_dense_name=None,
        min_units=4  # <-- increase minimum neurons per layer
    ):
    
    masks = {}
    for name, (A, G, V) in stats.items():
        if A.size == 0:
            masks[name] = np.array([], dtype=np.float32)
            continue

        def normalize(x):
            x2 = x - np.min(x)
            mx = np.max(x2)
            return x2 / (mx + 1e-12)

        # compute score
        score = alpha * normalize(A) + beta * normalize(G) + gamma * normalize(V)

        # ---- NEVER prune final Dense layer ----
        if last_dense_name is not None and name == last_dense_name:
            masks[name] = np.ones_like(score, dtype=np.float32)
            continue

        # ---- compute K ----
        k = int(len(score) * keep_ratio)
        k = max(k, min_units)        # enforce minimum
        k = min(k, len(score))       # safety

        # ---- If all scores are zero → randomly pick k neurons ----
        if np.all(score == 0):
            sel = np.random.choice(len(score), size=k, replace=False)
            mask = np.zeros_like(score)
            mask[sel] = 1
            masks[name] = mask.astype(np.float32)
            continue

        # normal path
        thresh = np.partition(score, -k)[-k]
        mask = (score >= thresh).astype(np.float32)

        # final safety: ensure >= min_units survive
        if np.sum(mask) < min_units:
            topk = np.argsort(score)[-min_units:]
            mask = np.zeros_like(score)
            mask[topk] = 1

        masks[name] = mask.astype(np.float32)

    return masks

# =====================================================================
# 5) DNAGate Layer (Zeroing)
# =====================================================================
# =====================================================================
# 5) DNAGate Layer (Zeroing) — NOW FULLY SERIALIZABLE
# =====================================================================
class DNAGate(tf.keras.layers.Layer):
    def __init__(self, mask=None, **kwargs):
        super().__init__(**kwargs)
        # store original numpy mask (for config saving)
        self._mask_np = None
        if mask is not None:
            self._mask_np = np.array(mask, dtype=np.float32)

        # keras-serializable field (converted to list)
        self.mask_list = self._mask_np.tolist() if self._mask_np is not None else None
        self.mask = None

    def build(self, input_shape):
        if self.mask_list is None:
            self.mask = None
        else:
            # rebuild tensor mask
            mask_np = np.array(self.mask_list, dtype=np.float32)
            if input_shape[-1] != mask_np.shape[0]:
                raise ValueError(
                    f"DNAGate mask length {mask_np.shape[0]} != layer units {input_shape[-1]}"
                )
            self.mask = tf.constant(mask_np, dtype=self.dtype)
        super().build(input_shape)

    def call(self, x):
        if self.mask is None:
            return x
        return x * self.mask

    # ⭐⭐ IMPORTANT — MAKES THE LAYER SAVEABLE ⭐⭐
    def get_config(self):
        config = super().get_config()
        config.update({
            "mask": self.mask_list,   # store mask as list
        })
        return config

    @classmethod
    def from_config(cls, config):
        # mask list gets restored automatically
        return cls(**config)

def build_masked_model(orig_model, masks):
    inp = Input(shape=orig_model.input_shape[1:])
    x = inp
    for layer in orig_model.layers:
        x = layer(x)
        if layer.name in masks:
            x = DNAGate(mask=masks[layer.name], name=layer.name + "_dna")(x)
    new_model = Model(inp, x)
    # copy weights where possible
    for l in orig_model.layers:
        try:
            new_model.get_layer(l.name).set_weights(l.get_weights())
        except Exception:
            pass
    return new_model

# =====================================================================
# 6) FLOPS + INFERENCE TIME
# =====================================================================
def dense_flops(in_size, out_size, bias=True):
    # multiply-adds counted as 2 ops per MAC
    return int(in_size * out_size * 2 + (out_size if bias else 0))

def model_flops(model):
    total = 0
    in_size = int(model.input_shape[1])
    for layer in model.layers:
        if isinstance(layer, Dense):
            total += dense_flops(in_size, layer.units, layer.use_bias)
            in_size = layer.units
    return total

def effective_flops(model, masks):
    total = 0
    in_size = int(model.input_shape[1])
    for layer in model.layers:
        if isinstance(layer, Dense):
            if layer.name in masks and masks[layer.name].size > 0:
                units = int(np.sum(masks[layer.name]))
            else:
                units = layer.units
            total += dense_flops(in_size, units, layer.use_bias)
            in_size = units
    return total

def flops_to_gflops(f):
    return f / 1e9

def flops_to_mflops(f):
    return f / 1e6

def measure_inference_time(model, X, runs=200):
    """Return average inference time in milliseconds per sample.
       Accepts X as pandas DataFrame or numpy array."""
    if isinstance(X, pd.DataFrame):
        X_np = X.to_numpy()
    else:
        X_np = X
    if len(X_np) == 0:
        return float("nan")
    # pick random indices
    idx = np.random.randint(0, len(X_np), size=min(runs, len(X_np)))
    samples = X_np[idx]
    # warm-up single predict
    model.predict(samples[:1], verbose=0)
    t0 = time.time()
    model.predict(samples, verbose=0)
    t1 = time.time()
    total_ms = (t1 - t0) * 1000.0
    return total_ms / len(samples)

# =====================================================================
# 7) STRUCTURAL PRUNING (REMOVE NEURONS) — protect final Dense layer
# =====================================================================
def structurally_prune_fnn(orig_model, masks, last_dense_name=None):
    inp = Input(shape=orig_model.input_shape[1:])
    x = inp
    prev_sel = None

    for layer in orig_model.layers:
        if isinstance(layer, Dense):
            W, b = layer.get_weights()

            # If previous layer was pruned → prune rows
            if prev_sel is not None:
                W = W[prev_sel, :]

            # ---- SAFETY: determine neurons to keep ----
            if last_dense_name is not None and layer.name == last_dense_name:
                # never prune final layer
                sel = np.arange(layer.units)

            else:
                mask = masks.get(layer.name, None)
                if mask is not None and mask.size > 0:
                    sel = np.where(mask == 1)[0]

                    # 💥 SAFETY FIX: ensure >=1 neuron survives
                    if len(sel) == 0:
                        sel = np.array([np.argmax(mask)])  
                else:
                    sel = None

            # ---- Apply pruning ----
            if sel is not None:
                W_new = W[:, sel]
                b_new = b[sel]
                new_units = len(sel)
            else:
                W_new = W
                b_new = b
                new_units = W.shape[1]

            new_layer = Dense(new_units, activation=layer.activation, name=layer.name)
            x = new_layer(x)
            new_layer.set_weights([W_new, b_new])

            prev_sel = sel

        else:
            x = layer(x)

    return Model(inp, x)

# =====================================================================
# 8) MAIN PIPELINE (auto-detect model type)
# =====================================================================
def fnn_pruning_pipeline(model_path, dataset_path,
                         keep_ratio=0.7,
                         alpha=0.5, beta=0.3, gamma=0.2,
                         calib_batches=30, batch_size=64, ft_epochs=3):
    # 1) load model and detect type
    print("Loading model:", model_path)
    model = load_model(model_path)
    model_type = detect_model_type(model)
    print(f"[INFO] Detected model type: {model_type}")

    loss_name, metrics = get_loss_and_metrics_for_type(model_type)
    print(f"[INFO] Using loss='{loss_name}', metrics={metrics}")

    # 2) load dataset adapted to model_type
    print("Loading dataset:", dataset_path)
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = load_any_csv_dataset(dataset_path, model_type=model_type)

    # 3) find dense layers and the last Dense layer name
    dense_layers = get_dense_layers(model)
    last_dense_name = dense_layers[-1].name if len(dense_layers) > 0 else None
    print("Dense layers:", [l.name for l in dense_layers], "| last_dense:", last_dense_name)

    # 4) compute stats (activation + gradient)
    print("Computing activation & gradient stats...")
    stats = compute_activation_grad_stats(model, dense_layers, X_train, y_train, model_type,
                                          batch_size=batch_size, max_batches=calib_batches)

    # 5) compute masks (protect final layer)
    masks = compute_importance_mask(stats, keep_ratio=keep_ratio, alpha=alpha, beta=beta, gamma=gamma, last_dense_name=last_dense_name)
    for k,v in masks.items():
        if v.size > 0:
            print(f"{k} -> kept units: {int(np.sum(v))}/{len(v)}")

    # 6) build masked model (DNAGate zeroing) and compile with correct loss/metrics
    print("Building masked model (DNAGate zeroing)...")
    masked_model = build_masked_model(model, masks)
    masked_model.compile(optimizer='adam', loss=loss_name, metrics=metrics)

    # compute baseline & masked flops/accuracy
    baseline_acc = None
    masked_acc_before = None
    if model_type == "regression":
        baseline_eval = model.evaluate(X_val, y_val, verbose=0)
        baseline_acc = baseline_eval[0] if len(baseline_eval)>0 else None
        masked_acc_before = masked_model.evaluate(X_val, y_val, verbose=0)[0]
    else:
        baseline_acc = model.evaluate(X_val, y_val, verbose=0)[1]  # accuracy or sparse_accuracy
        masked_acc_before = masked_model.evaluate(X_val, y_val, verbose=0)[1]

    baseline_flops = model_flops(model)
    masked_flops = effective_flops(model, masks)

    print(f"Baseline Acc: {baseline_acc} | FLOPS: {baseline_flops}")
    print(f"Masked Acc (before FT): {masked_acc_before} | Effective FLOPS: {masked_flops}")

    # 7) fine-tune masked model
    print("Fine-tuning masked model...")
    masked_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=ft_epochs, verbose=2)

    # evaluate masked after FT
    if model_type == "regression":
        masked_eval = masked_model.evaluate(X_val, y_val, verbose=0)
        masked_acc_after = masked_eval[0] if len(masked_eval)>0 else None
    else:
        masked_acc_after = masked_model.evaluate(X_val, y_val, verbose=0)[1]

    # 8) structural pruning (permanent neuron removal)
    print("Constructing structurally-pruned model (remove neurons)...")
    pruned_model = structurally_prune_fnn(model, masks, last_dense_name=last_dense_name)
    pruned_model.compile(optimizer='adam', loss=loss_name, metrics=metrics)

    print("Fine-tuning pruned model...")
    pruned_model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=ft_epochs, verbose=2)

    # final evals
    if model_type == "regression":
        pruned_eval = pruned_model.evaluate(X_val, y_val, verbose=0)
        pruned_acc = pruned_eval[0] if len(pruned_eval)>0 else None
    else:
        pruned_acc = pruned_model.evaluate(X_val, y_val, verbose=0)[1]

    pruned_flops = model_flops(pruned_model)

    # compute GFLOPS / MFLOPS / inference times
    baseline_gflops = flops_to_gflops(baseline_flops)
    masked_gflops = flops_to_gflops(masked_flops)
    pruned_gflops = flops_to_gflops(pruned_flops)

    baseline_mflops = flops_to_mflops(baseline_flops)
    masked_mflops = flops_to_mflops(masked_flops)
    pruned_mflops = flops_to_mflops(pruned_flops)

    baseline_time_ms = measure_inference_time(model, X_val)
    masked_time_ms = measure_inference_time(masked_model, X_val)
    pruned_time_ms = measure_inference_time(pruned_model, X_val)

    print("\n=== FINAL REPORT ===")
    print(f"Model type: {model_type}")
    print(f"Baseline:   Acc/metric={baseline_acc} | FLOPS={baseline_flops} | MFLOPS={baseline_mflops:.6f} | GFLOPS={baseline_gflops:.10f} | Time={baseline_time_ms:.4f} ms/sample")
    print(f"Masked:     Acc/metric={masked_acc_after} | EFLOPS={masked_flops} | MFLOPS={masked_mflops:.6f} | GFLOPS={masked_gflops:.10f} | Time={masked_time_ms:.4f} ms/sample")
    print(f"Pruned:     Acc/metric={pruned_acc} | FLOPS={pruned_flops} | MFLOPS={pruned_mflops:.6f} | GFLOPS={pruned_gflops:.10f} | Time={pruned_time_ms:.4f} ms/sample")
    if baseline_flops > 0:
        print(f"FLOPS reduction (pruned): {(baseline_flops - pruned_flops)/baseline_flops:.2%}")
    print("==============================================\n")

    print(f"Pruned:     Acc/metric={pruned_acc} | FLOPS={pruned_flops} | MFLOPS={pruned_mflops:.6f} | GFLOPS={pruned_gflops:.10f} | Time={pruned_time_ms:.4f} ms/sample")

    # ---- NEW METRIC REDUCTIONS ADDED BELOW ----
    if baseline_flops > 0:
        flop_reduction = (baseline_flops - pruned_flops) / baseline_flops
        gflop_reduction = (baseline_gflops - pruned_gflops) / baseline_gflops if baseline_gflops > 0 else 0
    else:
        flop_reduction = 0
        gflop_reduction = 0

    acc_reduction = baseline_acc - pruned_acc if baseline_acc is not None else None

    print(f"FLOPS reduction (pruned): {flop_reduction:.2%}")
    print(f"GFLOPS reduction (pruned): {gflop_reduction:.2%}")
    print(f"Accuracy reduction: {acc_reduction}")
    # ---- END NEW LINES ----

    print("==============================================\n")

    # save masked/pruned models next to original
    base = os.path.basename(model_path)
    root = os.path.splitext(base)[0]
    folder = os.path.dirname(model_path) or "."

    masked_path = os.path.join(folder, root + "_masked.h5")
    pruned_path = os.path.join(folder, root + "_pruned.h5")

    try:
        print(f"[INFO] Saving masked model to: {masked_path}")
        masked_model.save(masked_path)
    except Exception as e:
        print(f"[WARN] Failed to save masked model: {e}")

    try:
        print(f"[INFO] Saving pruned model to: {pruned_path}")
        pruned_model.save(pruned_path)
    except Exception as e:
        print(f"[WARN] Failed to save pruned model: {e}")

    # return X_val for further external checks if desired
    return model, masked_model, pruned_model, masks, X_val

# =====================================================================
# 9) RUN (example)
# =====================================================================
if __name__ == "__main__":
    # change these to your files
    model_path = r"D:\college\sem-8\models\dementia_fnn_model.h5"
    dataset_path = r"D:\college\sem-8\dataset\clean_dementia_dataset.csv"

    original_model, masked_model, pruned_model, masks, X_val = fnn_pruning_pipeline(
        model_path=model_path,
        dataset_path=dataset_path,
        keep_ratio=0.8,
        alpha=0.5, beta=0.3, gamma=0.2,
        calib_batches=30, batch_size=64, ft_epochs=15
    )


# **Main CNN Pruned**

In [ ]:
"""
sequential_cnn_pruning_full_fixed.py
Full Sequential CNN pruning pipeline (single-file)

- Loads user Sequential CNN (.h5/.keras / SavedModel)
- Sanitizes layer names in .h5 if they contain '/'
- Loads folder dataset (image_dataset_from_directory)
- Computes activation/gradient/variance importance
- Creates masks (keep_ratio)
- Builds masked model (gating layer)
- Fine-tunes masked model
- Structurally prunes model (conv filter removal, prune Dense outputs only)
- Computes FLOPS & timings, evaluates models
- Saves models and masks

Fixes:
- Ensures final layer matches dataset classes (automatically rebuilds final layer if needed)
- Uses appropriate loss function (binary vs sparse categorical) during stat collection and training
- Adds GFLOPS, accuracy reduction, inference timing
"""

import os
import json
import math
import time
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

# ---------------------------
# USER CONFIG (edit paths)
# ---------------------------
MODEL_PATH = r"D:\college\sem-8\models\garbage_cnn_model.h5"   # path to model (.h5/.keras or SavedModel dir)
DATASET_PATH = r"D:\college\sem-8\dataset\Garbage classification\Garbage classification"      # folder with subfolders per class
SAVE_DIR = r"pruning_output"                                 # where outputs are saved
base_name = os.path.splitext(os.path.basename(MODEL_PATH))[0]


KEEP_RATIO = 0.83            # fraction of channels/units to keep
ALPHA, BETA, GAMMA = 0.4, 0.3, 0.3  # importance weights
BATCH_SIZE = 64
CALIB_BATCHES = 30
FT_EPOCHS = 3
FT_BATCHES_TO_USE = 150
PLOT_RESULTS = True
VERBOSE = True

os.makedirs(SAVE_DIR, exist_ok=True)
tf.random.set_seed(42)
np.random.seed(42)

def log(*args):
    if VERBOSE:
        print("[INFO]", *args)

# ---------------------------
# Safe load model (handles '/' in h5 layer names)
# ---------------------------
def safe_load_model(model_path):
    """
    Try normal load_model; if fails (e.g., '/' in layer names in H5), sanitize model_config JSON and rebuild.
    Returns a Keras model (compiled=False).
    """
    log("Loading model:", model_path)
    # Try direct load first
    try:
        m = tf.keras.models.load_model(model_path, compile=False)
        log("Loaded model directly.")
        return m
    except Exception as e:
        log("Direct load failed:", e)

    # If HDF5, attempt to sanitize layer names in model_config
    try:
        with h5py.File(model_path, "r") as f:
            if "model_config" in f:
                raw = f["model_config"][()]
                if isinstance(raw, bytes):
                    raw = raw.decode("utf-8")
                cfg_json = json.loads(raw)
                changed = False
                for layer in cfg_json.get("config", {}).get("layers", []):
                    cfg = layer.get("config", {})
                    name = cfg.get("name")
                    if isinstance(name, str) and "/" in name:
                        new_name = name.replace("/", "_")
                        cfg["name"] = new_name
                        changed = True
                        log(f"[FIX] layer name: {name} -> {new_name}")
                if changed:
                    fixed_json = json.dumps(cfg_json)
                    model = tf.keras.models.model_from_json(fixed_json)
                    model.load_weights(model_path)
                    log("Loaded model from sanitized JSON + weights.")
                    return model
    except Exception as e2:
        log("H5 sanitization attempt failed:", e2)

    # final fallback: try load_model with safe_mode=False (older TF)
    try:
        m = tf.keras.models.load_model(model_path, compile=False, safe_mode=False)
        log("Loaded model with safe_mode=False.")
        return m
    except Exception as e3:
        log("All load attempts failed:", e3)
        raise RuntimeError("Failed to load model. Ensure path and format are correct.")

# ---------------------------
# Rebuild final layer to match num_classes (safe, best-effort)
# ---------------------------
def ensure_output_matches_dataset(orig_model, num_classes):
    """
    If the model's current output shape doesn't match num_classes, rebuild final output
    to match. Works for Sequential-style networks. Returns new_model, loss_fn.
    """
    # Determine model's output dim
    out_shape = tuple(orig_model.output_shape) if orig_model.output_shape is not None else None
    # If already matches (and for multiclass softmax case), return original and appropriate loss.
    if out_shape is not None:
        if num_classes == 2 and (out_shape[-1] == 1 or out_shape[-1] == 2):
            # binary case: allow Dense(1) or Dense(2) (Dense(2) could be softmax but labels are 0/1)
            log("Model output seems compatible with binary classification.")
            loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False) if out_shape[-1] == 1 else tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
            return orig_model, loss_fn
        if num_classes > 2 and out_shape[-1] == num_classes:
            log("Model output matches dataset classes.")
            loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
            return orig_model, loss_fn

    # Need to rebuild final layer
    log("Rebuilding model final layer to match num_classes:", num_classes)
    # We'll clone all layers except the last one and then append a new final dense.
    # Use layer.from_config to avoid reusing layer objects in two models.
    new_seq = models.Sequential(name=(orig_model.name or "rebuilt_model") + "_rebuilt")
    # Add InputLayer
    input_shape = orig_model.input_shape[1:]
    new_seq.add(layers.InputLayer(input_shape=tuple(input_shape)))

    # Clone all layers except the last (we will replace last)
    # We'll attempt to copy weights for layers that remain identical
    for layer in orig_model.layers[:-1]:
        try:
            cfg = layer.get_config()
            Cls = layer.__class__
            cloned = Cls.from_config(cfg)
            new_seq.add(cloned)
            # set weights if possible and shapes match
            try:
                w = layer.get_weights()
                if w:
                    new_seq.layers[-1].set_weights(w)
            except Exception:
                # ignore weight copy failures
                pass
        except Exception:
            # fallback: try to append original layer (may cause errors but best-effort)
            try:
                new_seq.add(layer)
            except Exception:
                log("Warning: couldn't clone layer", layer.name, "- skipping weights copy.")

    # Add new final layer depending on num_classes
    if num_classes == 2:
        # binary: Dense(1, activation='sigmoid')
        new_seq.add(layers.Dense(1, activation="sigmoid", name="output_rebuilt"))
        loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    else:
        new_seq.add(layers.Dense(num_classes, activation="softmax", name="output_rebuilt"))
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

    # Try to compile minimal to ensure shape correctness (we'll let caller compile fully later)
    return new_seq, loss_fn

# ---------------------------
# Dataset loader (folder-based)
# ---------------------------
def load_image_folder_dataset(path, image_size, batch_size=BATCH_SIZE):
    log("Loading dataset folder:", path, "image_size:", image_size)
    train_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=image_size,
        batch_size=batch_size,
    )
    rescaler = layers.Rescaling(1.0 / 255)
    train_ds = train_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.map(lambda x, y: (rescaler(x), y)).prefetch(tf.data.AUTOTUNE)
    return train_ds, val_ds

# ---------------------------
# Activation & gradient stats (uses provided loss function)
# ---------------------------
def compute_activation_grad_stats(model, layer_names, dataset, loss_fn, max_batches=CALIB_BATCHES):
    """
    For each layer in layer_names, compute:
      - A: mean(abs(activation)) per filter/unit
      - G: mean(abs(grad wrt activation)) per filter/unit
      - V: variance(activation) per filter/unit
    Returns: dict name -> (A, G, V)
    """
    log("Computing activation & gradient stats...")
    results = {n: [] for n in layer_names}
    grad_results = {n: [] for n in layer_names}
    var_results = {n: [] for n in layer_names}

    batch_count = 0
    for x_batch, y_batch in dataset:
        if batch_count >= max_batches:
            break
        batch_count += 1
        layer_acts = {}
        with tf.GradientTape(persistent=True) as tape:
            a = x_batch
            for layer in model.layers:
                a = layer(a)
                if layer.name in layer_names:
                    tape.watch(a)
                    layer_acts[layer.name] = a
            preds = a
            # compute loss using provided loss_fn (works for both binary & sparse categorical)
            # When loss_fn is a Keras loss instance, call like a function
            # Ensure y_batch dtype is int for sparse CE, float for binary BCE
            try:
                loss_vals = loss_fn(y_batch, preds)
            except Exception:
                # fallback attempt: convert types for binary
                if isinstance(loss_fn, tf.keras.losses.BinaryCrossentropy):
                    loss_vals = loss_fn(tf.cast(y_batch, tf.float32), preds)
                else:
                    loss_vals = loss_fn(y_batch, preds)
            loss = tf.reduce_mean(loss_vals)

        for name in layer_names:
            a = layer_acts[name]
            if len(a.shape) == 4:
                A = tf.reduce_mean(tf.abs(a), axis=(0,1,2)).numpy()
                V = tf.math.reduce_variance(a, axis=(0,1,2)).numpy()
            else:
                A = tf.reduce_mean(tf.abs(a), axis=0).numpy()
                V = tf.math.reduce_variance(a, axis=0).numpy()

            grad = tape.gradient(loss, a)
            if grad is None:
                G = np.zeros_like(A)
            else:
                if len(grad.shape) == 4:
                    G = tf.reduce_mean(tf.abs(grad), axis=(0,1,2)).numpy()
                else:
                    G = tf.reduce_mean(tf.abs(grad), axis=0).numpy()

            results[name].append(A)
            var_results[name].append(V)
            grad_results[name].append(G)
        del tape

    stats = {}
    for name in layer_names:
        A = np.mean(results[name], axis=0)
        V = np.mean(var_results[name], axis=0)
        G = np.mean(grad_results[name], axis=0)
        stats[name] = (A, G, V)
        log(f"{name}: len={len(A)} meanA={A.mean():.6e} meanG={G.mean():.6e} meanV={V.mean():.6e}")
    return stats

# ---------------------------
# Importance scores & masks
# ---------------------------
def compute_importance_scores(stats, alpha=ALPHA, beta=BETA, gamma=GAMMA):
    def normalize(x):
        x = x - x.min()
        if x.max() > 0:
            x = x / x.max()
        return x
    scores = {}
    for name, (A, G, V) in stats.items():
        scores[name] = alpha * normalize(A) + beta * normalize(G) + gamma * normalize(V)
    return scores

def make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO):
    masks = {}
    for name, score in score_map.items():
        k = max(1, int(len(score) * keep_ratio))
        thresh = np.partition(score, -k)[-k]
        mask = (score >= thresh).astype(np.float32)
        masks[name] = mask
        log(f"{name}: keep {int(mask.sum())}/{len(mask)}")
    return masks

# ---------------------------
# Mask gate layer & masked model
# ---------------------------
class CNNGate(tf.keras.layers.Layer):
    def __init__(self, channels, mask=None, **kwargs):
        """
        channels: int number of channels this gate controls
        mask: 1D array-like of length 'channels' with 0/1 values (or floats in [0,1]).
              If provided, gate variable is initialized to these values; otherwise ones.
        """
        super().__init__(**kwargs)
        self.channels = int(channels)
        # store mask as numpy array if provided (for serialization convenience)
        self._init_mask = None if mask is None else np.array(mask, dtype=np.float32)
        init_val = self._init_mask if self._init_mask is not None else np.ones((self.channels,), dtype=np.float32)
        # gate is non-trainable scalar per-channel multiplier
        self.gate = self.add_weight(
            name="gate",
            shape=(self.channels,),
            initializer=tf.keras.initializers.Constant(init_val),
            trainable=False,
            dtype=tf.float32,
        )

    def call(self, inputs):
        # supports inputs with shape [batch, H, W, C] or [batch, C] (works broadcasting)
        g = self.gate
        # expand dims to match channels in conv output
        if len(inputs.shape) == 4:
            return inputs * g[None, None, None, :]
        elif len(inputs.shape) == 2:
            return inputs * g[None, :]
        else:
            # fallback broadcasting
            return inputs * g

    def get_config(self):
        cfg = super().get_config()
        # do NOT embed gate values directly here (weights are saved by Keras),
        # but we include channels for reconstruction convenience.
        cfg.update({
            "channels": self.channels,
            # don't include mask/gate here to avoid duplicating weight data;
            # the gate weight will be saved/loaded normally by Keras.
        })
        return cfg

    @classmethod
    def from_config(cls, config):
        # config may only contain "channels" — weight values will be restored by Keras load.
        channels = config.pop("channels")
        return cls(channels=channels, **config)


def build_masked_model(orig_model, masks):
    """
    Build a new functional model with cloned layers from orig_model and insert a CNNGate
    after each layer named in `masks`. The gate is initialized from masks[layer_name].
    """
    log("Building masked model with gates (cloning layers)...")
    inp = tf.keras.Input(shape=orig_model.input_shape[1:])
    x = inp

    # keep mapping from original layer name -> new layer object (for weight copying)
    for layer in orig_model.layers:
        # clone layer if possible to avoid reusing original layer objects
        try:
            cfg = layer.get_config()
            Cls = layer.__class__
            new_layer = Cls.from_config(cfg)
        except Exception:
            # fallback: try to reuse the layer (less safe)
            new_layer = layer

        # apply the new layer to current tensor
        x = new_layer(x)

        # copy weights if layer had weights and we cloned it
        try:
            w = layer.get_weights()
            if w:
                try:
                    new_layer.set_weights(w)
                except Exception:
                    # some layers (e.g., fused ops) may not accept direct set_weights — ignore
                    pass
        except Exception:
            pass

        # insert gate if this original layer is in masks
        if layer.name in masks:
            mask = np.array(masks[layer.name], dtype=np.float32)
            channels = int(mask.shape[0])
            gate_layer = CNNGate(channels=channels, mask=mask, name=layer.name + "_gate")
            x = gate_layer(x)
            # gate weight is already initialized from mask in CNNGate.__init__, so nothing else to do

    masked = tf.keras.Model(inputs=inp, outputs=x, name=(orig_model.name or "model") + "_masked")
    log("Masked model created:", masked.name)
    return masked

# ---------------------------
# Structural pruning (safe)
# ---------------------------
def prune_structural_sequential(orig_model, masks, input_shape):
    """
    Structural pruning for Sequential models.
    - Prune Conv2D output filters using masks[layer.name]
    - Prune Dense output units only (do not slice Dense input rows that come from Flatten/Conv)
    - Attempt to slice BatchNorm params to match conv outputs
    """
    log("Structural pruning (safe) start...")
    new_layers = []
    prev_was_conv_like = False  # indicates that Flatten/Conv preceded Dense inputs

    for layer in orig_model.layers:
        # Conv2D: slice output channels
        if isinstance(layer, layers.Conv2D):
            W, b = layer.get_weights()
            orig_out = W.shape[-1]
            mask = masks.get(layer.name, np.ones(orig_out, dtype=np.float32))
            keep_idx = np.where(mask == 1)[0]
            if keep_idx.size == 0:
                keep_idx = np.array([int(np.argmax(mask))], dtype=np.int32)
            W_new = W[:, :, :, keep_idx]
            b_new = b[keep_idx]
            new_conv = layers.Conv2D(
                filters=len(keep_idx),
                kernel_size=layer.kernel_size,
                strides=layer.strides,
                padding=layer.padding,
                activation=layer.activation,
                use_bias=layer.use_bias,
                name=layer.name + "_pruned"
            )
            new_layers.append((new_conv, [W_new, b_new]))
            prev_was_conv_like = True
            continue

        # BatchNorm: slice params if prev was conv-like
        if isinstance(layer, layers.BatchNormalization):
            try:
                weights = layer.get_weights()
                if prev_was_conv_like and new_layers:
                    prev_layer_obj, prev_w = new_layers[-1]
                    if prev_w is not None:
                        out_ch = prev_w[0].shape[-1]  # kernel last dim
                        gamma, beta, mean, var = weights
                        gamma = gamma[:out_ch]
                        beta = beta[:out_ch]
                        mean = mean[:out_ch]
                        var = var[:out_ch]
                        new_bn = layers.BatchNormalization.from_config(layer.get_config())
                        new_layers.append((new_bn, [gamma, beta, mean, var]))
                        continue
            except Exception:
                pass
            # fallback keep BN as-is
            try:
                new_bn = layers.BatchNormalization.from_config(layer.get_config())
                new_layers.append((new_bn, layer.get_weights()))
            except Exception:
                new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
            # prev_was_conv_like unchanged
            continue

        # MaxPool/Activation/Dropout/Flatten/GlobalAvgPool: clone or reuse
        if isinstance(layer, (layers.MaxPooling2D, layers.Activation, layers.ReLU, layers.Dropout, layers.Flatten, layers.GlobalAveragePooling2D)):
            try:
                cloned = layer.__class__.from_config(layer.get_config())
                w = layer.get_weights() if hasattr(layer, "get_weights") else None
                new_layers.append((cloned, w if w else None))
            except Exception:
                new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
            if isinstance(layer, layers.Flatten) or isinstance(layer, layers.GlobalAveragePooling2D):
                prev_was_conv_like = True
            continue

        # Dense: prune outputs only (safe)
        if isinstance(layer, layers.Dense):
            W, b = layer.get_weights()  # shape (in_dim, out_dim)
            out_mask = masks.get(layer.name, np.ones(W.shape[1], dtype=np.float32))
            out_idx = np.where(out_mask == 1)[0]
            if out_idx.size == 0:
                out_idx = np.array([int(np.argmax(out_mask))], dtype=np.int32)
            W_new = W[:, out_idx]   # keep all input rows (safe)
            b_new = b[out_idx]
            new_dense = layers.Dense(units=W_new.shape[1], activation=layer.activation, name=layer.name + "_pruned")
            new_layers.append((new_dense, [W_new, b_new]))
            prev_was_conv_like = False
            continue

        # Fallback for other layers
        try:
            cloned = layer.__class__.from_config(layer.get_config())
            w = layer.get_weights() if hasattr(layer, "get_weights") else None
            new_layers.append((cloned, w if w else None))
        except Exception:
            new_layers.append((layer, layer.get_weights() if hasattr(layer, "get_weights") else None))
        prev_was_conv_like = False

    # Build new Sequential model with InputLayer
    seq = models.Sequential(name=orig_model.name + "_struct_pruned")
    seq.add(layers.InputLayer(input_shape=tuple(input_shape)))
    for lyr_obj, w in new_layers:
        seq.add(lyr_obj)
        if w is not None:
            try:
                seq.layers[-1].set_weights(w)
            except Exception as e:
                log("Warning: couldn't set weights for", seq.layers[-1].name, ":", e)
    log("Structural pruning complete. New model summary:")
    seq.summary()
    return seq

# ---------------------------
# FLOPS and timing helpers
# ---------------------------
def calculate_conv_flops(input_shape, kernel_shape, strides=(1,1), padding='same'):
    h_in, w_in, c_in = input_shape
    kh, kw, _, c_out = kernel_shape
    if padding == 'same':
        h_out = math.ceil(h_in / strides[0])
        w_out = math.ceil(w_in / strides[1])
    else:
        h_out = math.ceil((h_in - kh + 1) / strides[0])
        w_out = math.ceil((w_in - kw + 1) / strides[1])
    flops = h_out * w_out * (kh * kw * c_in) * c_out * 2
    return flops, (h_out, w_out, c_out)

def calculate_dense_flops(in_size, out_size):
    return in_size * out_size * 2

def calculate_model_flops(model, input_shape):
    total = 0
    current_shape = tuple(input_shape)
    for layer in model.layers:
        if isinstance(layer, layers.Conv2D):
            weights = layer.get_weights()
            if not weights:
                continue
            kernel_shape = weights[0].shape  # (kh, kw, in_c, out_c)
            layer_flops, current_shape = calculate_conv_flops(current_shape, kernel_shape, strides=layer.strides, padding=layer.padding)
            total += layer_flops
        elif isinstance(layer, layers.Flatten):
            current_shape = (int(np.prod(current_shape)),)
        elif isinstance(layer, layers.Dense):
            in_size = current_shape[0] if isinstance(current_shape, tuple) and len(current_shape)>0 else int(current_shape)
            layer_flops = calculate_dense_flops(in_size, layer.units)
            total += layer_flops
            current_shape = (layer.units,)
        elif isinstance(layer, layers.MaxPooling2D):
            h,w,c = current_shape
            pool = layer.pool_size[0] if hasattr(layer.pool_size, "__getitem__") else layer.pool_size
            current_shape = (h//pool, w//pool, c)
        else:
            # ignore other layers for shape changes
            pass
    return total

def measure_inference_time(model, sample_batch, steps=20):
    model.predict(sample_batch, verbose=0)  # warmup
    t0 = time.time()
    for _ in range(steps):
        model.predict(sample_batch, verbose=0)
    t1 = time.time()
    return (t1 - t0) / steps

# ---------------------------
# Save masks helper
# ---------------------------
def save_masks(masks, path):
    serial = {k: v.tolist() for k,v in masks.items()}
    with open(path, "w") as f:
        json.dump(serial, f, indent=2)
    log("Saved masks to", path)

# ---------------------------
# Plot helper
# ---------------------------
def plot_mask_histograms(masks, outdir=SAVE_DIR):
    if not PLOT_RESULTS:
        return
    for name, mask in masks.items():
        plt.figure(figsize=(5,2))
        plt.title(name)
        plt.hist(mask, bins=2)
        plt.xlabel("0=pruned, 1=kept")
        plt.tight_layout()
        plt.savefig(os.path.join(outdir, f"mask_{name}.png"))
        plt.close()

# ---------------------------
# MAIN pipeline
# ---------------------------
def full_pipeline(model_path, dataset_path):
    # 1) load model (safe)
    model = safe_load_model(model_path)
    log("Model loaded. Summary:")
    model.summary()

    # 2) load dataset and infer num_classes
    # We need input size so infer from model
    input_shape = model.input_shape[1:]
    log("Inferred input shape:", input_shape)

    train_ds, val_ds = load_image_folder_dataset(dataset_path, image_size=input_shape[:2], batch_size=BATCH_SIZE)
    # Determine number of classes from dataset
    try:
        # tf.data.Dataset from image_dataset_from_directory has .class_names on the original Dataset returned object,
        # but not on the batched dataset — so inspect via a fresh loader:
        tmp = tf.keras.utils.image_dataset_from_directory(dataset_path, image_size=input_shape[:2], batch_size=1)
        num_classes = len(tmp.class_names)
        del tmp
    except Exception:
        # fallback: infer from labels in train_ds
        classes = set()
        for _, y in train_ds.take(10):
            classes.update(y.numpy().tolist())
        num_classes = max(classes) + 1 if classes else 2

    log("Detected dataset classes (num_classes):", num_classes)

    # 3) ensure model output matches dataset classes
    model, loss_fn = ensure_output_matches_dataset(model, num_classes)
    # compile original so evaluate works (use small lr default) with detected loss
    if isinstance(loss_fn, tf.keras.losses.BinaryCrossentropy):
        model.compile(optimizer=tf.keras.optimizers.Adam(), loss=loss_fn, metrics=["accuracy"])
    else:
        model.compile(optimizer=tf.keras.optimizers.Adam(), loss=loss_fn, metrics=["accuracy"])

    # reprint summary
    log("Final model used (after potential rebuild). Summary:")
    model.summary()

    # calibration subset for stats
    calib_ds = train_ds.take(CALIB_BATCHES)

    # sample for timing
    try:
        sample_x, _ = next(iter(val_ds))
    except Exception:
        sample_x, _ = next(iter(train_ds))
    sample_x_small = sample_x[:min(16, sample_x.shape[0])]

    # 4) choose layers to prune (conv + hidden dense only, not final output)
    dense_layers = [lyr for lyr in model.layers if isinstance(lyr, layers.Dense)]
    last_dense = dense_layers[-1] if dense_layers else None

    prune_layer_names = []
    for lyr in model.layers:
        if isinstance(lyr, layers.Conv2D):
            prune_layer_names.append(lyr.name)
        elif isinstance(lyr, layers.Dense) and lyr is not last_dense:
            prune_layer_names.append(lyr.name)
    log("Layers considered for pruning:", prune_layer_names)

    # 5) compute stats (pass loss_fn)
    stats = compute_activation_grad_stats(model, prune_layer_names, calib_ds, loss_fn=loss_fn, max_batches=CALIB_BATCHES)

    # 6) importance & masks
    score_map = compute_importance_scores(stats)
    masks = make_masks_from_scores(score_map, keep_ratio=KEEP_RATIO)
    save_masks(masks, os.path.join(SAVE_DIR, "masks.json"))
    plot_mask_histograms(masks)

    # 7) build masked model and compile (use same loss)
    masked_model = build_masked_model(model, masks)
    masked_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=loss_fn, metrics=["accuracy"])
    log("Masked model built.")

    # quick eval before FT (small subset)
        # quick eval before FT (small subset)
    try:
        loss0, acc0 = masked_model.evaluate(val_ds.take(5), verbose=0)
        log("Masked model pre-FT acc:", acc0)
    except Exception as e:
        log("Masked model pre-eval failed:", e)
        acc0 = None

    # 8) measure baseline flops & time
    baseline_flops = calculate_model_flops(model, input_shape)
    baseline_time = measure_inference_time(model, sample_x_small, steps=10)
    log(f"Baseline FLOPS: {baseline_flops:,}, baseline time (avg batch): {baseline_time:.4f}s")

    # 9) fine-tune masked model
    try:
        masked_model.fit(
            train_ds.take(FT_BATCHES_TO_USE),
            validation_data=val_ds.take(5),
            epochs=FT_EPOCHS,
            verbose=2
        )
    except Exception as e:
        log("Masked fine-tune failed/partial:", e)

    # 10) structural prune
    # NOTE: you can prune the original model or the masked_model.
    #       pruning masked_model preserves gating decisions — often desired.
    try:
        pruned_model = prune_structural_sequential(masked_model, masks, input_shape)
    except Exception:
        # fallback to pruning original model if pruning masked_model fails
        pruned_model = prune_structural_sequential(model, masks, input_shape)

    # compile pruned model for evaluation / training with same loss
    pruned_model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss=loss_fn,
        metrics=["accuracy"]
    )

    log("Starting fine-tuning of PRUNED model (EarlyStopping)...")
    early_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=2,
        restore_best_weights=True
    )
    try:
        pruned_model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=20,
            batch_size=BATCH_SIZE,
            callbacks=[early_cb],
            verbose=2
        )
    except Exception as e:
        log("Pruned model fine-tune failed/partial:", e)

    # 11) evaluate models
    log("Evaluating Original model:")
    orig_loss, orig_acc = model.evaluate(val_ds, verbose=0)
    log("Original acc:", orig_acc)

    log("Evaluating Masked model (after FT):")
    try:
        mask_loss, mask_acc = masked_model.evaluate(val_ds, verbose=0)
        log("Masked acc:", mask_acc)
    except Exception as e:
        log("Masked evaluate failed:", e)
        mask_acc = None

    log("Evaluating Pruned model:")
    try:
        pruned_loss, pruned_acc = pruned_model.evaluate(val_ds, verbose=0)
        log("Pruned acc:", pruned_acc)
    except Exception as e:
        log("Pruned evaluate failed:", e)
        pruned_acc = None

    # 12) FLOPS & timing after prune
    pruned_flops = calculate_model_flops(pruned_model, input_shape)
    pruned_time = measure_inference_time(pruned_model, sample_x_small, steps=10)
    log(f"Pruned FLOPS: {pruned_flops:,}, pruned time: {pruned_time:.4f}s")

    # 13) summary & save
    reduction = 1.0 - (pruned_flops / baseline_flops) if baseline_flops > 0 else 0.0
    log("="*60)
    log("SUMMARY:")
    log(f"Baseline FLOPS: {baseline_flops:,}")
    log(f"Pruned FLOPS: {pruned_flops:,}")
    log(f"FLOPS reduction: {reduction:.2%}")
    log(f"Original acc: {orig_acc}, Masked acc: {mask_acc}, Pruned acc: {pruned_acc}")
    log("="*60)

    # ---- Extra Metrics: GFLOPS + Accuracy Reduction ----
    baseline_gflops = baseline_flops / 1e9
    pruned_gflops = pruned_flops / 1e9
    gflops_reduction = 1.0 - (pruned_gflops / baseline_gflops) if baseline_gflops > 0 else 0.0
    acc_reduction = (orig_acc - pruned_acc) if (orig_acc is not None and pruned_acc is not None) else None

    log(f"Baseline GFLOPS: {baseline_gflops:.4f}")
    log(f"Pruned GFLOPS: {pruned_gflops:.4f}")
    log(f"GFLOPS reduction: {gflops_reduction:.2%}")
    if acc_reduction is not None:
        log(f"Accuracy reduction: {acc_reduction:.4f}")
    else:
        log("Accuracy reduction: N/A")

    # save artifacts
    try:
        # use proper file extensions to avoid Keras save errors
        baseline_name = base_name + "_baseline.keras"
        masked_name   = base_name + "_masked.keras"
        pruned_name   = base_name + "_pruned.keras"

        model.save(os.path.join(SAVE_DIR, baseline_name))
        masked_model.save(os.path.join(SAVE_DIR, masked_name))
        pruned_model.save(os.path.join(SAVE_DIR, pruned_name))

        log(f"Saved models under names: {baseline_name}, {masked_name}, {pruned_name}")

        log("Saved models to", SAVE_DIR)
    except Exception as e:
        log("Save models failed:", e)

    return {
        "model": model,
        "masked_model": masked_model,
        "pruned_model": pruned_model,
        "masks": masks,
        "baseline_flops": baseline_flops,
        "pruned_flops": pruned_flops,
        "baseline_time": baseline_time,
        "pruned_time": pruned_time,
        "orig_acc": orig_acc,
        "mask_acc": mask_acc,
        "pruned_acc": pruned_acc
    }

# ---------------------------
# Run
# ---------------------------
if __name__ == "__main__":
    out = full_pipeline(MODEL_PATH, DATASET_PATH)
    log("Pipeline finished. Outputs:", out.keys())


# **Main Resnet**

In [ ]:
# ==========================================================
# RESNET50 SOFT CHANNEL MASKING (BN-SAFE, GRAPH-SAFE, SERIALIZABLE)
# ==========================================================

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# ---------------- CONFIG ----------------
MODEL_PATH   = "/content/ishan_100__resnet.h5"
DATASET_PATH = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"

BATCH = 32
KEEP_RATIO = 0.85
ALPHA = 0.1
EPOCHS = 10
PATIENCE = 3
UNFREEZE_LAST = 15
img_size=(128,128)
# ---------------- DATA ----------------
def load_ds(path, img_size):
    train = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=img_size,
        batch_size=BATCH
    )
    val = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=img_size,
        batch_size=BATCH
    )

    scale = layers.Rescaling(1. / 255)
    train = train.map(lambda x, y: (scale(x), y)).prefetch(tf.data.AUTOTUNE)
    val   = val.map(lambda x, y: (scale(x), y)).prefetch(tf.data.AUTOTUNE)
    return train, val

# ---------------- FIND CONVS ----------------
def find_convs(model):
    convs = []

    def walk(l):
        if isinstance(l, layers.Conv2D):
            convs.append(l)
        if isinstance(l, tf.keras.Model):
            for x in l.layers:
                walk(x)

    walk(model)
    return convs

# ---------------- SOFT CHANNEL MASK ----------------
@tf.keras.utils.register_keras_serializable()
class ChannelMask(layers.Layer):
    def __init__(self, mask, alpha=0.1, **kwargs):
        super().__init__(**kwargs)
        mask = np.asarray(mask, dtype="float32")
        mask = np.where(mask > 0, 1.0, alpha)
        mask = mask / np.mean(mask)
        self.mask_init = mask
        self.alpha = alpha

    def build(self, input_shape):
        self.mask = self.add_weight(
            name="mask",
            shape=(input_shape[-1],),
            initializer=tf.constant_initializer(self.mask_init),
            trainable=False
        )

    def call(self, x):
        return x * self.mask[None, None, None, :]

    def get_config(self):
        cfg = super().get_config()
        cfg.update({
            "mask": self.mask_init.tolist(),
            "alpha": self.alpha
        })
        return cfg

    @classmethod
    def from_config(cls, config):
        mask = config.pop("mask")
        return cls(mask=mask, **config)

# ---------------- MASK CREATION ----------------
def make_masks(convs):
    masks = {}
    for l in convs:
        W = l.kernel.numpy()
        scores = np.mean(np.abs(W), axis=(0, 1, 2))
        k = max(1, int(len(scores) * KEEP_RATIO))
        thr = np.partition(scores, -k)[-k]
        mask = (scores >= thr).astype("float32")
        masks[l.name] = mask
        print(f"[PRUNE] {l.name}: keep {int(mask.sum())}/{len(mask)}")
    return masks

# ---------------- GRAPH-SAFE MASK INSERT ----------------
def build_masked_resnet(base_model, masks):
    tensor_map = {}
    last_tensor = None

    for inp in base_model.inputs:
        tensor_map[inp] = inp
        last_tensor = inp

    for layer in base_model.layers:

        if isinstance(layer, layers.InputLayer):
            continue

        inbound = layer.input
        if isinstance(inbound, list):
            mapped = [tensor_map.get(x, x) for x in inbound]
        else:
            mapped = tensor_map.get(inbound, inbound)

        x = layer(mapped)

        # Insert mask AFTER BatchNorm
        if isinstance(layer, layers.BatchNormalization):
            prev = layer.input._keras_history[0]
            if isinstance(prev, layers.Conv2D) and prev.name in masks:
                x = ChannelMask(
                    masks[prev.name],
                    alpha=ALPHA,
                    name=prev.name + "_mask"
                )(x)

        tensor_map[layer.output] = x
        last_tensor = x

    return tf.keras.Model(
        inputs=base_model.inputs,
        outputs=last_tensor,
        name=base_model.name + "_masked"
    )


# ---------------- FLOPs ----------------
def baseline_flops(model):
    total = 0
    for l in find_convs(model):
        if not l._inbound_nodes:
            continue
        _, h, w, cout = l.output.shape
        cin = l.input.shape[-1]
        k = l.kernel_size[0]
        total += h * w * cin * cout * k * k * 2
    return int(total)

def effective_flops_from_masks(model, masks):
    total = 0
    for l in find_convs(model):
        if l.name in masks and l._inbound_nodes:
            _, h, w, _ = l.output.shape
            cin = l.input.shape[-1]
            k = l.kernel_size[0]
            active = int(np.sum(masks[l.name]))
            total += h * w * cin * active * k * k * 2
    return int(total)

# ---------------- MAIN ----------------
def run():
    print("[INFO] Loading base model...")
    base = tf.keras.models.load_model(MODEL_PATH, compile=False)

    # 🔧 force base graph build
    dummy = tf.zeros((1,) + base.input_shape[1:])
    _ = base(dummy)

    train, val = load_ds(DATASET_PATH, base.input_shape[1:3])

    convs = find_convs(base)
    masks = make_masks(convs)

    masked = build_masked_resnet(base, masks)

    # 🔧 force masked graph build
    _ = masked(dummy)

    # freeze
    for l in masked.layers[:-UNFREEZE_LAST]:
        l.trainable = False

    masked.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    early = tf.keras.callbacks.EarlyStopping(
        patience=PATIENCE,
        restore_best_weights=True,
        monitor="val_loss"
    )

    masked.fit(
        train,
        validation_data=val,
        epochs=EPOCHS,
        callbacks=[early],
        verbose=1
    )

    masked.save("masked_finetuned_resnet_leaf.keras")

    b = baseline_flops(base)
    e = effective_flops_from_masks(base, masks)

    print("\n=========== EFFECTIVE FLOPs ANALYSIS ===========")
    print(f"Baseline FLOPs : {b:,}")
    print(f"Effective FLOPs: {e:,}")
    print(f"Reduction (%)  : {(b - e) / b * 100:.2f}%")
    print("===============================================")

# ---------------- RUN ----------------
if __name__ == "__main__":
    run()


# **Main EfficientNet**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# ---------------- CONFIG ----------------
MODEL_PATH   = "/content/efficientnet_fruit_f.keras"
DATASET_PATH = "/kaggle/input/fruits/fruits-360_100x100/fruits-360/Training"

BATCH = 32
KEEP_RATIO = 0.85
ALPHA = 0.1
EPOCHS = 10
PATIENCE = 3
UNFREEZE_LAST = 20
IMG_SIZE = (128, 128)

# ---------------- DATA ----------------
def load_ds(path, img_size):
    train = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=img_size,
        batch_size=BATCH
    )
    val = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=img_size,
        batch_size=BATCH
    )

    scale = layers.Rescaling(1. / 255)
    train = train.map(lambda x, y: (scale(x), y)).prefetch(tf.data.AUTOTUNE)
    val   = val.map(lambda x, y: (scale(x), y)).prefetch(tf.data.AUTOTUNE)
    return train, val


# ---------------- FIND PRUNABLE CONVS ----------------
# EfficientNet rule:
# ✔ mask Conv2D with kernel_size=(1,1)
# ✘ skip DepthwiseConv2D
def find_prunable_convs(model):
    convs = []

    def walk(l):
        if isinstance(l, layers.Conv2D):
            if l.kernel_size == (1, 1):
                convs.append(l)
        if isinstance(l, tf.keras.Model):
            for x in l.layers:
                walk(x)

    walk(model)
    return convs


# ---------------- SOFT CHANNEL MASK ----------------
@tf.keras.utils.register_keras_serializable()
class ChannelMask(layers.Layer):
    def __init__(self, mask, alpha=0.1, **kwargs):
        super().__init__(**kwargs)
        mask = np.asarray(mask, dtype="float32")
        mask = np.where(mask > 0, 1.0, alpha)
        mask = mask / np.mean(mask)   # keep activation scale stable
        self.mask_init = mask
        self.alpha = alpha

    def build(self, input_shape):
        self.mask = self.add_weight(
            name="mask",
            shape=(input_shape[-1],),
            initializer=tf.constant_initializer(self.mask_init),
            trainable=False
        )

    def call(self, x):
        return x * self.mask[None, None, None, :]

    def get_config(self):
        cfg = super().get_config()
        cfg.update({
            "mask": self.mask_init.tolist(),
            "alpha": self.alpha
        })
        return cfg

    @classmethod
    def from_config(cls, config):
        mask = config.pop("mask")
        return cls(mask=mask, **config)


# ---------------- MASK CREATION ----------------
def make_masks(convs):
    masks = {}
    for l in convs:
        W = l.kernel.numpy()
        scores = np.mean(np.abs(W), axis=(0, 1, 2))  # per-output-channel
        k = max(1, int(len(scores) * KEEP_RATIO))
        thr = np.partition(scores, -k)[-k]
        mask = (scores >= thr).astype("float32")
        masks[l.name] = mask
        print(f"[PRUNE] {l.name}: keep {int(mask.sum())}/{len(mask)}")
    return masks


# ---------------- GRAPH-SAFE MASK INSERT ----------------
def build_masked_efficientnet(base_model, masks):
    tensor_map = {}
    last_tensor = None

    for inp in base_model.inputs:
        tensor_map[inp] = inp
        last_tensor = inp

    for layer in base_model.layers:

        if isinstance(layer, layers.InputLayer):
            continue

        inbound = layer.input
        if isinstance(inbound, list):
            mapped = [tensor_map.get(x, x) for x in inbound]
        else:
            mapped = tensor_map.get(inbound, inbound)

        x = layer(mapped)

        # Insert mask AFTER BatchNorm if previous layer was Conv2D(1x1)
        if isinstance(layer, layers.BatchNormalization):
            prev = layer.input._keras_history[0]
            if isinstance(prev, layers.Conv2D):
                if prev.name in masks:
                    x = ChannelMask(
                        masks[prev.name],
                        alpha=ALPHA,
                        name=prev.name + "_mask"
                    )(x)

        tensor_map[layer.output] = x
        last_tensor = x

    return tf.keras.Model(
        inputs=base_model.inputs,
        outputs=last_tensor,
        name=base_model.name + "_masked"
    )


# ---------------- FLOPs ----------------
# EfficientNet FLOPs: count ONLY pointwise convs
def baseline_flops(model):
    total = 0
    for l in find_prunable_convs(model):
        if not l._inbound_nodes:
            continue
        _, h, w, cout = l.output.shape
        cin = l.input.shape[-1]
        total += h * w * cin * cout * 2
    return int(total)


def effective_flops_from_masks(model, masks):
    total = 0
    for l in find_prunable_convs(model):
        if l.name in masks and l._inbound_nodes:
            _, h, w, _ = l.output.shape
            cin = l.input.shape[-1]
            active = int(np.sum(masks[l.name]))
            total += h * w * cin * active * 2
    return int(total)


# ---------------- MAIN ----------------
def run():
    print("[INFO] Loading base EfficientNet...")
    base = tf.keras.models.load_model(MODEL_PATH, compile=False)

    # 🔧 force graph build
    dummy = tf.zeros((1,) + base.input_shape[1:])
    _ = base(dummy)

    train, val = load_ds(DATASET_PATH, base.input_shape[1:3])

    convs = find_prunable_convs(base)
    masks = make_masks(convs)

    masked = build_masked_efficientnet(base, masks)

    # 🔧 force masked graph build
    _ = masked(dummy)

    # freeze most layers
    for l in masked.layers[:-UNFREEZE_LAST]:
        l.trainable = False

    masked.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    early = tf.keras.callbacks.EarlyStopping(
        patience=PATIENCE,
        restore_best_weights=True,
        monitor="val_loss"
    )

    masked.fit(
        train,
        validation_data=val,
        epochs=EPOCHS,
        callbacks=[early],
        verbose=1
    )

    masked.save("masked_finetuned_efficientnet_fruit_image_classification.keras")

    b = baseline_flops(base)
    e = effective_flops_from_masks(base, masks)

    print("\n=========== EFFECTIVE FLOPs ANALYSIS ===========")
    print(f"Baseline FLOPs : {b:,}")
    print(f"Effective FLOPs: {e:,}")
    print(f"Reduction (%)  : {(b - e) / b * 100:.2f}%")
    print("===============================================")


# ---------------- RUN ----------------
if __name__ == "__main__":
    run()


# **Main MobileNet**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# ---------------- CONFIG ----------------
MODEL_PATH   = "/content/mobilenet_f.keras"
DATASET_PATH = "/root/.cache/kagglehub/datasets/puneet6060/intel-image-classification/versions/2/seg_train/seg_train"


BATCH = 32
KEEP_RATIO = 0.85
ALPHA = 0.1
EPOCHS = 10
PATIENCE = 3
UNFREEZE_LAST = 20
IMG_SIZE = (128, 128)

# ---------------- DATA ----------------
def load_ds(path, img_size):
    train = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=img_size,
        batch_size=BATCH
    )
    val = tf.keras.utils.image_dataset_from_directory(
        path,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=img_size,
        batch_size=BATCH
    )

    scale = layers.Rescaling(1. / 255)
    train = train.map(lambda x, y: (scale(x), y)).prefetch(tf.data.AUTOTUNE)
    val   = val.map(lambda x, y: (scale(x), y)).prefetch(tf.data.AUTOTUNE)
    return train, val


# ---------------- FIND PRUNABLE CONVS ----------------
# MobileNet rule:
# ✔ Conv2D kernel=(1,1)  (pointwise)
# ✘ DepthwiseConv2D
def find_prunable_convs(model):
    convs = []

    def walk(l):
        if isinstance(l, layers.Conv2D):
            if l.kernel_size == (1, 1):
                convs.append(l)
        if isinstance(l, tf.keras.Model):
            for x in l.layers:
                walk(x)

    walk(model)
    return convs


# ---------------- SOFT CHANNEL MASK ----------------
@tf.keras.utils.register_keras_serializable()
class ChannelMask(layers.Layer):
    def __init__(self, mask, alpha=0.1, **kwargs):
        super().__init__(**kwargs)
        mask = np.asarray(mask, dtype="float32")
        mask = np.where(mask > 0, 1.0, alpha)
        mask = mask / np.mean(mask)  # activation scale stability
        self.mask_init = mask
        self.alpha = alpha

    def build(self, input_shape):
        self.mask = self.add_weight(
            name="mask",
            shape=(input_shape[-1],),
            initializer=tf.constant_initializer(self.mask_init),
            trainable=False
        )

    def call(self, x):
        return x * self.mask[None, None, None, :]

    def get_config(self):
        cfg = super().get_config()
        cfg.update({
            "mask": self.mask_init.tolist(),
            "alpha": self.alpha
        })
        return cfg

    @classmethod
    def from_config(cls, config):
        mask = config.pop("mask")
        return cls(mask=mask, **config)


# ---------------- MASK CREATION ----------------
def make_masks(convs):
    masks = {}
    for l in convs:
        W = l.kernel.numpy()
        scores = np.mean(np.abs(W), axis=(0, 1, 2))
        k = max(1, int(len(scores) * KEEP_RATIO))
        thr = np.partition(scores, -k)[-k]
        mask = (scores >= thr).astype("float32")
        masks[l.name] = mask
        print(f"[PRUNE] {l.name}: keep {int(mask.sum())}/{len(mask)}")
    return masks


# ---------------- GRAPH-SAFE MASK INSERT ----------------
def build_masked_mobilenet(base_model, masks):
    tensor_map = {}

    for inp in base_model.inputs:
        tensor_map[inp] = inp

    for layer in base_model.layers:

        if isinstance(layer, layers.InputLayer):
            continue

        inbound = layer.input
        if isinstance(inbound, list):
            mapped = [tensor_map.get(x, x) for x in inbound]
        else:
            mapped = tensor_map.get(inbound, inbound)

        x = layer(mapped)

        # Insert mask AFTER BatchNorm if previous layer was Conv2D(1x1)
        if isinstance(layer, layers.BatchNormalization):
            prev = layer.input._keras_history[0]
            if isinstance(prev, layers.Conv2D):
                if prev.kernel_size == (1, 1) and prev.name in masks:
                    x = ChannelMask(
                        masks[prev.name],
                        alpha=ALPHA,
                        name=prev.name + "_mask"
                    )(x)

        tensor_map[layer.output] = x

    return tf.keras.Model(
        inputs=base_model.inputs,
        outputs=x,
        name=base_model.name + "_masked"
    )


# ---------------- FLOPs ----------------
# Count ONLY pointwise convs
def baseline_flops(model):
    total = 0
    for l in find_prunable_convs(model):
        if not l._inbound_nodes:
            continue
        _, h, w, cout = l.output.shape
        cin = l.input.shape[-1]
        total += h * w * cin * cout * 2
    return int(total)


def effective_flops_from_masks(model, masks):
    total = 0
    for l in find_prunable_convs(model):
        if l.name in masks and l._inbound_nodes:
            _, h, w, _ = l.output.shape
            cin = l.input.shape[-1]
            active = int(np.sum(masks[l.name]))
            total += h * w * cin * active * 2
    return int(total)


# ---------------- MAIN ----------------
def run():
    print("[INFO] Loading base MobileNet...")
    base = tf.keras.models.load_model(MODEL_PATH, compile=False)

    # force graph build
    dummy = tf.zeros((1,) + base.input_shape[1:])
    _ = base(dummy)

    train, val = load_ds(DATASET_PATH, base.input_shape[1:3])

    convs = find_prunable_convs(base)
    masks = make_masks(convs)

    masked = build_masked_mobilenet(base, masks)

    # force masked graph build
    _ = masked(dummy)

    for l in masked.layers[:-UNFREEZE_LAST]:
        l.trainable = False

    masked.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    early = tf.keras.callbacks.EarlyStopping(
        patience=PATIENCE,
        restore_best_weights=True,
        monitor="val_loss"
    )

    masked.fit(
        train,
        validation_data=val,
        epochs=EPOCHS,
        callbacks=[early],
        verbose=1
    )

    masked.save("masked_finetuned_mobilenet_Intel.keras")

    b = baseline_flops(base)
    e = effective_flops_from_masks(base, masks)

    print("\n=========== EFFECTIVE FLOPs ANALYSIS ===========")
    print(f"Baseline FLOPs : {b:,}")
    print(f"Effective FLOPs: {e:,}")
    print(f"Reduction (%)  : {(b - e) / b * 100:.2f}%")
    print("===============================================")


# ---------------- RUN ----------------
if __name__ == "__main__":
    run()


# **Main Transformer**

In [ ]:
"""
UNIVERSAL TRANSFORMER ATTENTION HEAD PRUNING + MASKED FINE-TUNING
(FINAL, GUARANTEED-RUNNING VERSION)

✔ Any CSV or JSON Lines dataset
✔ Label = last column fallback
✔ Labels aligned to model output
✔ Binary + multiclass supported
✔ Fresh TextVectorization
✔ Soft attention-head pruning
✔ Masked fine-tuning (graph-safe)
✔ Correct FLOPs / Effective FLOPs
✔ Handles nested Transformer blocks
✔ Legacy .h5 compatible
"""

# ==========================================================
# IMPORTS
# ==========================================================

import json
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# ==========================================================
# CONFIG
# ==========================================================

MAX_LEN     = 200
VOCAB_SIZE = 20000
BATCH_SIZE = 64
EPOCHS     = 5
KEEP_RATIO = 0.7

# ==========================================================
# CUSTOM LAYERS (LEGACY SAFE)
# ==========================================================

@tf.keras.utils.register_keras_serializable(package="Custom")
class PositionalEmbedding(layers.Layer):
    def __init__(
        self,
        max_len=None,
        vocab_size=None,
        embed_dim=None,
        maxlen=None,          # legacy support
        **kwargs
    ):
        super().__init__(**kwargs)

        if max_len is None and maxlen is not None:
            max_len = maxlen

        if max_len is None:
            raise ValueError("max_len or maxlen must be provided")

        self.max_len = int(max_len)
        self.vocab_size = int(vocab_size)
        self.embed_dim = int(embed_dim)

        self.token_emb = layers.Embedding(self.vocab_size, self.embed_dim)
        self.pos_emb   = layers.Embedding(self.max_len, self.embed_dim)

    def call(self, x):
        pos = tf.range(start=0, limit=tf.shape(x)[-1])
        return self.token_emb(x) + self.pos_emb(pos)

    def get_config(self):
        cfg = super().get_config()
        cfg.update({
            "maxlen": self.max_len,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim
        })
        return cfg


@tf.keras.utils.register_keras_serializable(package="Custom")
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate

        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.ln1 = layers.LayerNormalization(epsilon=1e-6)
        self.ln2 = layers.LayerNormalization(epsilon=1e-6)
        self.drop1 = layers.Dropout(rate)
        self.drop2 = layers.Dropout(rate)

        self.last_attn = None

    def call(self, x, training=None):
        attn = self.att(x, x, training=training)
        self.last_attn = attn
        x = self.ln1(x + self.drop1(attn, training=training))
        ffn = self.ffn(x, training=training)
        return self.ln2(x + self.drop2(ffn, training=training))

    def get_config(self):
        cfg = super().get_config()
        cfg.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "rate": self.rate
        })
        return cfg


TransformerBlock = TransformerEncoder

# ==========================================================
# MASKED TRANSFORMER
# ==========================================================

class MaskedTransformer(tf.keras.Model):
    def __init__(self, base_model, masks):
        super().__init__()
        self.base = base_model
        self.masks = masks

    def call(self, inputs, training=None):
        x = inputs
        for layer in self.base.layers:
            if isinstance(layer, tf.keras.layers.InputLayer):
                continue

            if isinstance(layer, TransformerEncoder):
                attn = layer.att(x, x, training=training)

                if layer in self.masks:
                    H = layer.att.num_heads
                    D = attn.shape[-1]
                    Hd = D // H
                    m = self.masks[layer]

                    attn = tf.reshape(attn, (-1, tf.shape(attn)[1], H, Hd))
                    attn = attn * m[None, None, :, None]
                    attn = tf.reshape(attn, (-1, tf.shape(attn)[1], D))

                x = layer.ln1(x + layer.drop1(attn, training=training))
                x = layer.ln2(x + layer.drop2(layer.ffn(x), training=training))
            else:
                x = layer(x, training=training)

        return x

# ==========================================================
# DATASET LOADER (ALIGNED TO MODEL)
# ==========================================================

def load_text_dataset(path, model_num_outputs):

    if path.endswith(".csv"):
        df = pd.read_csv(path)
    elif path.endswith(".json"):
        rows = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                rows.append(json.loads(line))
        df = pd.DataFrame(rows)
    else:
        raise ValueError("Only CSV and JSON supported")

    print("[INFO] Dataset columns:", list(df.columns))

    text_cols = [c for c in df.columns if df[c].dtype == object]
    texts = df[text_cols].fillna("").agg(" ".join, axis=1).values

    label_col = df.columns[-1]
    print(f"[INFO] Using label column: {label_col}")

    labels = df[label_col].values

    if pd.api.types.is_numeric_dtype(labels):
        y_int = labels.astype(int)
    else:
        y_int = LabelEncoder().fit_transform(labels)

    # ======================================================
    # CRITICAL: BINARY vs MULTICLASS HANDLING
    # ======================================================

    if model_num_outputs == 1:
        # Dense(1) → sigmoid → binary
        y = y_int.astype("float32")
        loss = "binary_crossentropy"

    else:
        # Dense(N) → softmax
        y_int = np.clip(y_int, 0, model_num_outputs - 1)
        y = to_categorical(y_int, num_classes=model_num_outputs)
        loss = "categorical_crossentropy"

    return texts, y, loss

# ==========================================================
# TEXT VECTORIZATION
# ==========================================================

def build_vectorizer(texts):
    vec = layers.TextVectorization(
        max_tokens=VOCAB_SIZE,
        output_mode="int",
        output_sequence_length=MAX_LEN
    )
    vec.adapt(texts)
    return vec

# ==========================================================
# STRIP TRANSFORMER
# ==========================================================

def strip_transformer(model):
    if model.input.dtype in (tf.int32, tf.int64):
        return model

    for layer in model.layers:
        try:
            if layer.input.dtype in (tf.int32, tf.int64):
                return tf.keras.Model(layer.input, model.output)
        except:
            pass

    token_input = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32)
    return tf.keras.Model(token_input, model(token_input))

# ==========================================================
# UTILITIES
# ==========================================================

def get_all_layers(model):
    out = []
    for l in model.layers:
        out.append(l)
        if isinstance(l, tf.keras.Model):
            out.extend(get_all_layers(l))
    return out

# ==========================================================
# ATTENTION STATS + FLOPs
# ==========================================================

def compute_attention_head_stats(model, vectorizer, texts, max_batches=20):
    blocks = [l for l in get_all_layers(model) if isinstance(l, TransformerEncoder)]
    stats = {b: [] for b in blocks}

    ds = tf.data.Dataset.from_tensor_slices(texts).batch(BATCH_SIZE)
    for i, xb in enumerate(ds):
        if i >= max_batches:
            break

        tokens = tf.cast(vectorizer(xb), tf.int32)
        _ = model(tokens, training=False)

        for block in blocks:
            if block.last_attn is None:
                continue
            attn = block.last_attn
            H = block.att.num_heads
            D = attn.shape[-1]
            Hd = D // H
            reshaped = tf.reshape(attn, (-1, attn.shape[1], H, Hd))
            score = tf.reduce_mean(tf.abs(reshaped), axis=[0, 1, 3]).numpy()
            stats[block].append(score)

    return {k: np.mean(v, axis=0) for k, v in stats.items()}


def compute_importance_mask(stats, keep_ratio):
    masks = {}
    for block, score in stats.items():
        k = max(1, int(len(score) * keep_ratio))
        th = np.partition(score, -k)[-k]
        masks[block] = (score >= th).astype(np.float32)
    return masks


def attention_flops(seq_len, embed_dim):
    return 4 * seq_len * embed_dim * embed_dim + 2 * seq_len * seq_len * embed_dim


def transformer_model_flops(model):
    flops = 0
    for l in get_all_layers(model):
        if isinstance(l, TransformerEncoder):
            D = l.att.key_dim * l.att.num_heads
            flops += attention_flops(MAX_LEN, D)
    return flops


def effective_transformer_flops(model, masks):
    flops = 0
    for l in get_all_layers(model):
        if isinstance(l, TransformerEncoder):
            D = l.att.key_dim * l.att.num_heads
            H = l.att.num_heads
            kept = int(np.sum(masks.get(l, np.ones(H))))
            flops += attention_flops(MAX_LEN, D) * (kept / H)
    return flops

# ==========================================================
# MAIN PIPELINE
# ==========================================================

def universal_transformer_pruning(model_path, dataset_path):

    model = tf.keras.models.load_model(
        model_path,
        custom_objects={
            "PositionalEmbedding": PositionalEmbedding,
            "TransformerEncoder": TransformerEncoder,
            "TransformerBlock": TransformerEncoder,
        },
        compile=False
    )

    model_num_outputs = model.output_shape[-1]
    print(f"[INFO] Model output units: {model_num_outputs}")

    texts, y, loss_fn = load_text_dataset(dataset_path, model_num_outputs)

    Xtr, Xv, ytr, yv = train_test_split(
        texts, y,
        test_size=0.2,
        random_state=42,
        stratify=y if model_num_outputs == 1 else y.argmax(axis=1)
    )

    vectorizer = build_vectorizer(Xtr)
    transformer = strip_transformer(model)

    stats = compute_attention_head_stats(transformer, vectorizer, Xtr)
    masks = compute_importance_mask(stats, KEEP_RATIO)

    masked_model = MaskedTransformer(transformer, masks)
    masked_model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss=loss_fn,
        metrics=["accuracy"],
        run_eagerly=True
    )

    masked_model.fit(
        tf.cast(vectorizer(Xtr), tf.int32), ytr,
        validation_data=(tf.cast(vectorizer(Xv), tf.int32), yv),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE
    )

    acc = masked_model.evaluate(
        tf.cast(vectorizer(Xv), tf.int32), yv, verbose=0
    )[1]

    orig = transformer_model_flops(transformer)
    eff  = effective_transformer_flops(transformer, masks)

    print("\n=========== FINAL RESULTS ===========")
    print(f"Masked Accuracy     : {acc:.4f}")
    print(f"Original GFLOPs     : {orig / 1e9:.3f}")
    print(f"Effective GFLOPs    : {eff / 1e9:.3f}")
    print(f"FLOPs Reduction (%) : {(1 - eff / orig) * 100:.2f}%")

    masked_model.save("soft_pruned_transformer_imbd.keras")
    print("[INFO] Saved pruned model: soft_pruned_transformer.keras")

# ==========================================================
# RUN
# ==========================================================

if __name__ == "__main__":
    universal_transformer_pruning(
        r"D:\college\sem-8\final\custom_transformer_imdb.h5",
        r"D:\college\sem-8\dataset\tranformer dataset\IMDB Dataset.csv"
    )
